In [ ]:
import numpy as np
import random
import time
import matplotlib.pyplot as plt
import pickle

In [ ]:
!pip install gymnasium[atari]==0.27.1 -q
!pip install gymnasium[accept-rom-license] -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 45.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.7/434.7 kB 21.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
import gymnasium as gym
env = gym.make("ALE/KungFuMaster-v5", render_mode="rgb_array", obs_type = "grayscale")

/usr/local/lib/python3.9/dist-packages/jax/_src/deprecations.py:51: DeprecationWarning: jax.numpy.DeviceArray is deprecated. Use jax.Array.
  warnings.warn(message, DeprecationWarning)
/usr/local/lib/python3.9/dist-packages/jax/_src/deprecations.py:51: DeprecationWarning: jax.numpy.DeviceArray is deprecated. Use jax.Array.
  warnings.warn(message, DeprecationWarning)
/usr/local/lib/python3.9/dist-packages/gymnasium/utils/passive_env_checker.py:35: UserWarning: WARN: A Box observation space has an unconventional shape (neither an image, nor a 1D vector). We recommend flattening the observation to have only a 1D vector or use a custom policy to properly process the data. Actual observation shape: (210, 160)
  logger.warn(


In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Activation, Flatten
from keras.optimizers import Adam
from collections import deque
import tensorflow as tf
from tensorflow import keras

In [ ]:
update_counter = 0
N = 20
eps_start = 1
eps_decay_rate = 0.88
eps_min = 0.5

learning_rate = 0.001
gamma = 0.98
batch_size = 128
#buffer_size = 10_000
obs_dim = env.observation_space.shape
action_dim = env.action_space.n
#buffer = 1000
tf.keras.backend.set_image_data_format('channels_last')
trained_model = None

In [ ]:
class DQNAgent:
  def __init__(self, action_dim, learning_rate, gamma, batch_size, trained_model):
    self.learning_rate = learning_rate
    self.gamma = gamma
    self.batch_size = batch_size
    #self.buffer_size = buffer_size
    #self.obs_dim = obs_dim
    self.action_dim = action_dim

    self.replay_memory = deque(maxlen=512)


    if trained_model == None:
      self.model = self.create_model()
      self.target_model = self.model

    else:
      self.model = trained_model
      self.target_model = self.model

    #self.model = self.create_model()

    #self.target_model = self.model


  def create_model(self):

    model = Sequential()
    #model.add(Flatten())
    model.add(Conv2D(filters=256, kernel_size=(5, 5), activation='relu'))
    # model.add(Conv2D(1024, 5, 5))
    model.add(MaxPooling2D(pool_size=(4, 4)))
    model.add(Dropout(0.2))
    model.add(Flatten())

    model.add(Dense(128, activation='relu'))
    #model.add(Dense(128, activation='relu'))
    #model.add(Flatten())
    model.add(Dense(env.action_space.n, activation='softmax'))
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mse', metrics=['accuracy'])

    #print(model.summary())
    return model

  def select_action(self,epsilon, curr_obs, model):
    if random.random() <= epsilon:
      return random.randint(0, action_dim-1)
    else:
      #curr_obs = np.expand_dims(curr_obs, axis=3)
      curr_obs = tf.reshape(curr_obs,[1,210,160,1],)
      #print(f'shape of curr_obs{curr_obs.shape}')
      q_vals = model.predict(curr_obs)
      #q_vals = model.predict(np.expand_dims(curr_obs, 0))
      return np.argmax(q_vals[0])



  def memorize(self, curr_obs, action, reward, next_obs, terminated):
    self.replay_memory.append((curr_obs, action, reward, next_obs, terminated))



  def train(self):
    if len(self.replay_memory) < self.batch_size:
      return

    mini_batch = random.sample(self.replay_memory, self.batch_size)

    for curr_obs, action, reward, next_obs, terminated in mini_batch:
      curr_obs = tf.reshape(curr_obs,[1,210,160,1],)
      next_obs = tf.reshape(next_obs,[1,210,160,1],)
      #print(f'shape of curr_obs ,dtype in train() {next_obs.shape}, {next_obs.dtype}')
      if not terminated:
        target = reward + gamma* np.amax(self.target_model.predict(next_obs)[0])
      else:
        target = reward

      curr_qval = self.model.predict(curr_obs)
      #curr_qval = self.model.predict(np.expand_dims(curr_obs,0))
      curr_qval[0][action] = target


      #print(f'in train()\n curr_qval -> {curr_qval}')
      #print(f'in train()\n curr_qval shape -> {curr_qval.shape}')


      x, y = list(),list()

      x.append(curr_obs[0])
      y.append(curr_qval[0])


      self.model.fit(np.array(x), np.array(y), batch_size=self.batch_size, epochs=10, verbose=0)


  def update_model(self):
    global update_counter
    self.target_model.set_weights(self.model.get_weights())
    update_counter = 0



  def dqn(agent, episodes=200):
    scores = []
    epsilon = eps_start
    for episode in range(episodes):
      print('Episode - ', episode+1)

      curr_obs = env.reset()
      _obs = curr_obs[0]
      _obs = tf.reshape(_obs,[1,210,160,1],)
      _obs = tf.cast(_obs,dtype=tf.float32)

      #print(f'type of obs in dqn{_obs.dtype}')

      #_obs = tf.expand_dims(_obs, axis=1)
      #print(_obs.shape)
      #_obs = tf.reshape(_obs, (-1, 210, 160, 1))
      #print(_obs.shape)

      terminated = False
      truncated = False

      score = 0
      timestep = 0

      #i = 0
      while not (terminated):
        action = agent.select_action(epsilon, _obs, agent.model)

        next_obs, reward, terminated, truncated, info= env.step(action)
        next_obs = tf.cast(next_obs,dtype=tf.float32)
        #print(f'shape of next_obs {next_obs.shape}')

        #i+=1
        # print(f'next_obs {next_obs.shape}')
        # next_obs = next_obs.reshape(next_obs, [1, agent.obs_dim])
        # show_obs = cv2_
      # print(f'num of steps ==> {i}')
        #if i%1000 == 0:
        # cv2_imshow(next_obs)

        #next_obs = np.expand_dims(np.asarray(next_obs, dtype=np.float32), axis=0)


        agent.memorize(_obs, action, reward, next_obs, terminated)


        _obs = next_obs
        score += reward
        timestep +=1


      if len(agent.replay_memory) > batch_size:
          agent.train()

      if episode % 50 == 0:
        agent.model.save(f'cnn{episode}.h5')

      global update_counter, N
      if update_counter > N:
        agent.update_model()


      print(f"Exploration rate = {epsilon},", end = "\t")
      #print(epsilon*eps_decay_rate)
      epsilon = max(eps_min, epsilon*eps_decay_rate)


      print(f"completed with {timestep} timesteps, score = {score}", end = '\n')
      scores.append(score)



In [ ]:
agent = DQNAgent(action_dim, learning_rate, gamma, batch_size, trained_model)
#dqn(agent, model)
agent.dqn()
agent.model.save('DQN_CNN_200.h5')

Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 20ms/step
Exploration rate = 0.5,	completed with 906 timesteps, score = 0.0
Episode -  98
1/1 [==============================] - 0s 23ms/step
Exploration rate = 0.5,	completed with 886 timesteps, score = 0.0
Episode -  99
1/1 [==============================] - 0s 20ms/step
Exploration rate = 0.5,	completed with 732 timesteps, score = 0.0
Episode -  100
1/1 [==============================] - 0s 21ms/step
Exploration rate = 0.5,	completed with 679 timesteps, score = 0.0
Episode -  101
1/1 [==============================] - 0s 22ms/step
Exploration rate = 0.5,	completed with 842 timesteps, score = 0.0
Episode -  102
1/1 [==============================] - 0s 18ms/step
Exploration rate = 0.5,	completed with 858 timesteps, score = 0.0
Episode -  103
1/1 [==============================] - 0s 19ms/step
Exploration rate = 0.5,	completed with 792 timesteps, score = 0.0
Episode -  104
1/1 [===============

KeyboardInterrupt: ignored

In [ ]:
trained_model = keras.models.load_model('DQN_CNN_100.h5')

observation = env.reset()
_obs = observation[0]
_obs = tf.reshape(_obs,[1,210,160,1],)
_obs = tf.cast(_obs,dtype=tf.float32)

terminated = False
truncated = False
scores = 0
ts = 0
i = 1

'''action = env.action_space.sample()
_obs, reward, terminated, truncated, info = env.step(action)
print("Action taken - ", action)'''

while not terminated:
  #action = np.argmax(trained_model.predict(_obs)[0])
  q_vals = trained_model.predict(_obs)
  action = np.argmax(q_vals[0])

  #action = env.action_space.sample()
  print("Action taken - ", action)
  print("Prediction - ", trained_model.predict(_obs)[0])


  next_obs, reward, terminated, truncated, info = env.step(action)
  next_obs = tf.cast(_obs, dtype=tf.float32)
  _obs = next_obs

  ts += 1
  #env.render()
  scores += reward
  if ts == 1000:
   break

print('Total Score = ', scores)



1/1 [==============================] - 0s 495ms/step
Action taken -  9
1/1 [==============================] - 0s 117ms/step
Prediction -  [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
1/1 [==============================] - 0s 118ms/step
Action taken -  9
1/1 [==============================] - 0s 93ms/step
Prediction -  [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
1/1 [==============================] - 0s 77ms/step
Action taken -  9
1/1 [==============================] - 0s 74ms/step
Prediction -  [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
1/1 [==============================] - 0s 74ms/step
Action taken -  9
1/1 [==============================] - 0s 89ms/step
Prediction -  [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
1/1 [==============================] - 0s 71ms/step
Action taken -  9
1/1 [==============================] - 0s 76ms/step
Prediction -  [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
1/1 [==============================] - 0s 76ms/step
Action taken -  9
1/1 [======================

KeyboardInterrupt: ignored